# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
path = "../weather_data.csv"

weather_data = pd.read_csv(path)
weather_data

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Wind Speed,Cloudiness,Country,Data Receiving Time (GMT)
0,0,pirai,-22.63,-43.90,80.60,61,14.99,40,BR,1603739036
1,1,nanortalik,60.14,-45.24,36.05,84,8.84,100,GL,1603738704
2,2,geraldton,-28.77,114.60,55.40,93,10.25,40,AU,1603738900
3,3,barrow,71.29,-156.79,17.60,92,3.36,75,US,1603738501
4,4,carhuamayo,-10.92,-76.03,53.91,49,15.10,100,PE,1603739036
...,...,...,...,...,...,...,...,...,...,...
567,567,arona,28.10,-16.68,73.00,64,23.04,20,ES,1603739087
568,568,bandarbeyla,9.49,50.81,76.78,76,15.95,98,SO,1603738916
569,569,nkowakowa,-23.89,30.29,66.20,52,16.11,0,ZA,1603739087
570,570,jawhar,2.78,45.50,79.20,79,9.55,100,SO,1603739087


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

location = weather_data[["Latitude", "Longitude"]].astype(float)

humidity = weather_data['Humidity']

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(location, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1.4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
hotel_df = weather_data.loc[(weather_data['Max Temperature'] < 80) &
                                (weather_data['Max Temperature'] > 70) &
                                (weather_data['Wind Speed'] < 5) &
                                (weather_data['Cloudiness'] == 0) &
                                (weather_data['Humidity'] < 90)]
hotel_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Wind Speed,Cloudiness,Country,Data Receiving Time (GMT)
71,71,riyadh,24.69,46.72,73.40,28,3.38,0,SA,1603738749
82,82,bilma,18.69,12.92,73.44,21,2.37,0,NE,1603738924
199,199,antsohihy,-14.88,47.99,78.31,71,0.60,0,MG,1603739052
204,204,baripada,21.93,86.72,72.05,45,4.25,0,IN,1603739053
219,219,mankera,31.39,71.44,70.30,23,2.77,0,PK,1603739054
242,242,mersin,36.86,34.65,73.40,49,2.24,0,TR,1603739056
342,342,mubi,10.27,13.26,78.48,30,4.14,0,NG,1603739066
463,463,bosaso,11.28,49.18,75.88,62,3.85,0,SO,1603739078
505,505,karwar,14.80,74.13,77.00,85,3.91,0,IN,1603739081
526,526,nueve de julio,-35.44,-60.88,71.08,40,2.44,0,AR,1603739084


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df['Hotel Name'] = ""

params = {
    "radius": 5000,
    "types": "lodging",
    "language": "en",
    "key": g_key
}
    
for index, row in hotel_df.iterrows():
    
    lat = row["Latitude"]
    lng = row["Longitude"]

    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except:
        print("Missing field/result... skipping.")
hotel_df

C:\Users\austi\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\austi\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Wind Speed,Cloudiness,Country,Data Receiving Time (GMT),Hotel Name
71,71,riyadh,24.69,46.72,73.40,28,3.38,0,SA,1603738749,InterContinental Riyadh
82,82,bilma,18.69,12.92,73.44,21,2.37,0,NE,1603738924,Expeditions Ténére voyages
199,199,antsohihy,-14.88,47.99,78.31,71,0.60,0,MG,1603739052,Hotel de Ville
204,204,baripada,21.93,86.72,72.05,45,4.25,0,IN,1603739053,Aparna Hotel
219,219,mankera,31.39,71.44,70.30,23,2.77,0,PK,1603739054,Malik Jahanzaib house
242,242,mersin,36.86,34.65,73.40,49,2.24,0,TR,1603739056,Nostalgia Boutique Hotel
342,342,mubi,10.27,13.26,78.48,30,4.14,0,NG,1603739066,Hotel
463,463,bosaso,11.28,49.18,75.88,62,3.85,0,SO,1603739078,Red Sea Hotel
505,505,karwar,14.80,74.13,77.00,85,3.91,0,IN,1603739081,Netalco Connects
526,526,nueve de julio,-35.44,-60.88,71.08,40,2.44,0,AR,1603739084,Grand Hotel Libertad - GHL


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [8]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(location, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1.4)
hotel_layer = gmaps.marker_layer(
    locations, info_box_content= hotel_info
)


fig.add_layer(hotel_layer)

fig.add_layer(heat_layer)

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))